In [39]:
import urllib.request
import tarfile
import os
from pylab import *
%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import cross_val_score

from io import StringIO 
import pydot
from IPython.display import Image
from sklearn import tree

In [30]:
def get_data_and_target(df, features, target):
    return {"data":df[features],"target":df[target]}

In [18]:
def train_test_split_pd(data, target, test_size):
    mask = np.random.rand(len(data)) < test_size
    return data[mask], data[~mask], target[mask], target[~mask]

In [7]:
df0 = pd.read_csv("data/QN_merged.csv", index_col=0)

In [23]:
df0.head()

,total,0,10,15,20,25,30,35,40,45,...,200,ZIP,year,State,CountyName,rent0,rent1,rent2,rent3,rent4
0,9059,244,161,156,331,232,177,397,230,247,...,1062,11001,2011,36,Queens County,1300,1390,1570,1950,2150
1,5131,149,221,163,130,252,271,165,109,199,...,244,11004,2011,36,Queens County,1130,1200,1360,1690,1870
2,1356,22,42,85,76,74,54,34,68,39,...,141,11005,2011,36,Queens County,1180,1260,1420,1760,1950
3,13438,230,313,281,509,499,219,455,380,344,...,1701,11040,2011,36,Queens County,1440,1530,1730,2140,2370
4,9910,1306,732,481,740,601,513,392,431,382,...,303,11101,2011,36,Queens County,1120,1190,1350,1670,1850


In [29]:
temp = get_data_and_target(df0, ["rent0", "rent1", "rent2"], "200")

NameError: name 'data' is not defined

In [33]:
len(temp["data"])

129

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split_pd(temp["data"], temp["target"], 0.7)

In [38]:
len(Y_train)

90

In [40]:
def fit_tree(X, y, depth=10):
    ##: Using the DecisionTreeRegressor, train a model to depth 10.
    decision_tree = tree.DecisionTreeRegressor(max_depth=depth)
    return decision_tree.fit(X,y)

def display_tree(tmodel):
    dot_data = StringIO() 
    tree.export_graphviz(tmodel, out_file=dot_data) 
    graph = pydot.graph_from_dot_data(dot_data.getvalue())
    return Image(graph.create_png())

In [ ]:
tmodel = fit_tree(train_basic_features, train_basic_response, 5)
display_tree(tmodel)

In [ ]:
pd.scatter_matrix(train[features], figsize=(16,9))
plt.show()